# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv('Pokemon.csv')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [8]:
# Verify that Total equals the sum of HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed for every Pokémon
import pandas as pd

# Use the existing pokemon DataFrame if available; otherwise load it from CSV
try:
    df = pokemon.copy()
except NameError:
    df = pd.read_csv('Pokemon.csv')

# Calculate the total of the six ability fields
stat_sum = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].sum(axis=1)

# Check whether this calculated sum equals the provided Total field
all_match = (stat_sum == df['Total']).all()
print("All records match the formula:", all_match)

# Show the formula explicitly
print("Formula: Total = HP + Attack + Defense + Sp. Atk + Sp. Def + Speed")

# If any rows do not match, display a few examples
if not all_match:
    mismatch_df = df[stat_sum != df['Total']].copy()
    mismatch_df['Calculated_Total'] = stat_sum[stat_sum != df['Total']]
    print(mismatch_df[['Name', 'Total', 'Calculated_Total']].head())

All records match the formula: True
Formula: Total = HP + Attack + Defense + Sp. Atk + Sp. Def + Speed


In [7]:
# your code here

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed.

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](https://education-team-2020.s3.eu-west-1.amazonaws.com/ds-ai/lab-dataframe-calculation-and-transformation/images/one-hot-encoding.png)

In [4]:
# One-hot encode Type 1 and Type 2 columns to create numerical columns for each unique pokemon type
import pandas as pd

# Load dataset if not already loaded
try:
    df = pokemon.copy()
except NameError:
    df = pd.read_csv('Pokemon.csv')

# Fill missing values in 'Type 2' with a placeholder to include them in encoding
df['Type 2'] = df['Type 2'].fillna('None')

# Get unique types from both columns
unique_types = sorted(set(df['Type 1']).union(set(df['Type 2'])))

# Create binary columns for each type; 1 if pokemon has that type in either column, else 0
for t in unique_types:
    df[t] = ((df['Type 1'] == t) | (df['Type 2'] == t)).astype(int)

# Assign back to pokemon for later use
pokemon = df

# Display first few rows of the encoded columns to verify
pokemon[unique_types + ['Total']].head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,None,Normal,Poison,Psychic,Rock,Steel,Water,Total
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,318
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,405
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,525
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,625
4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,309


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [5]:
# Calculate correlation between Total and each encoded pokemon type column
# Ensure that 'unique_types' list exists (from the encoding step)
correlations = pokemon[unique_types].corrwith(pokemon['Total']).sort_values(ascending=False)

# Display correlations sorted in descending order
print(correlations)

# Identify the type with the highest correlation to Total
top_type = correlations.index[0]
print("Type with highest correlation to Total:", top_type)

Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
None       -0.185945
dtype: float64
Type with highest correlation to Total: Dragon


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [6]:
# Determine the best combination of Type 1 and Type 2 to maximize the average Total
import pandas as pd

# Use the encoded pokemon DataFrame; if not defined, load it again
try:
    df = pokemon.copy()
except NameError:
    df = pd.read_csv('Pokemon.csv')

# Replace missing Type 2 values with 'None' for pairing
df['Type 2'] = df['Type 2'].fillna('None')

# Create a tuple representing the unordered pair of types for each pokemon
df['type_pair'] = df.apply(lambda row: tuple(sorted([row['Type 1'], row['Type 2']])), axis=1)

# Calculate the average Total for each type pair
avg_total_by_pair = df.groupby('type_pair')['Total'].mean().sort_values(ascending=False)

# Display the average Total by pair
print(avg_total_by_pair)

# Identify the best type pair (highest average Total)
best_pair = avg_total_by_pair.index[0]
print("Best type combination to choose:", best_pair)

type_pair
(Dragon, Ice)        686.666667
(Dragon, Ghost)      680.000000
(Dragon, Steel)      680.000000
(Dragon, Fire)       657.000000
(Dragon, Psychic)    650.000000
                        ...    
(Fairy, Water)       335.000000
(Fairy, Normal)      330.000000
(Bug, None)          289.705882
(Bug, Water)         269.000000
(Bug, Ghost)         236.000000
Name: Total, Length: 133, dtype: float64
Best type combination to choose: ('Dragon', 'Ice')
